# Entity Extraction using IBM Watson Corpus

Assign libraries 

Uses Watson Natural language understanding for Entity Extraction, pandas for data cleaning  

#### Import libraries

In [1]:
import watson_developer_cloud
import watson_developer_cloud.natural_language_understanding.features.v1 as features
from watson_developer_cloud import ConversationV1
import pandas as pd
import gensim
import json

C:\Users\ssaha3237\AppData\Local\Continuum\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


#### Creating Dictionary for Indents used in IBM Watson Conversation API 

#### Creates a connection between Conversation API and python code and get all the Indents in the workspace

In [2]:
conversation = ConversationV1(
  username= "6128bc95-5e0f-4948-832e-34368882edbd",
  password= "j8yl15f0m6Wc",
  version= '2017-02-03'
)

In [17]:
def create_Dictionary():
    # --- Get all the Indents in the workspace -------------------------------------------------------------------------
    response_intent = conversation.request(method='GET',
                            url='/v1/workspaces/d29a1fab-4f32-4005-9558-df47eee29cdf/intents/',
                            params = {'version': '2017-02-03'})
    
    json_intent = json.loads(response_intent.text)
    
    # ---- List all the intents and gets all the sub intents related to that intent -------------------------------------
    list_indent = [str(json_intent['intents'][i]['intent']) for i in range(len(json_intent['intents']))]
    
    # ---- Create the dictionary for all indent -------------------------------------------------------------------------
    dictionary = {}
    for i in range(len(list_indent)) :
        response_supporting_entity = conversation.request(method='GET',
                                url='/v1/workspaces/d29a1fab-4f32-4005-9558-df47eee29cdf/intents/'+list_indent[i]+'/examples',
                                params = {'version': '2017-02-03'})

        json_supporting_entity = json.loads(response_supporting_entity.text)

        list_supporting_entity = [str(json_supporting_entity['examples'][j]['text']) for j in range(len(json_supporting_entity['examples']))]
        dictionary[list_indent[i]] = list_supporting_entity
        
    return dictionary

#### Getting indents and Entities from IBM watson and  calling the "create_Dictionary" function 

In [4]:
dictionary = create_Dictionary()
dictionary

{'goodbye': ['Bye',
  'Farewell',
  'Goodbye',
  'I am leaving.',
  "I'm leaving",
  'Sayonara',
  'See you later'],
 'hello': ['good morning',
  'greetings',
  'hello',
  'hi',
  'howdy',
  'I am here',
  'I have arrived'],
 'time': ['Give me the time.',
  'Is it late?',
  'Tell me the hour of the day.',
  'What is the current time?',
  'What time is it?',
  'What time of day is it']}

#### Call Watson using username and password provided by IBM Watson services 

In [5]:
nlu = watson_developer_cloud.NaturalLanguageUnderstandingV1(version='2017-02-27',
                                                            username='378d2ad7-9322-4e44-8b65-3b1e9fa19a81',
                                                            password='wPtkALoy6iJN')

#### Class for Analysing Questions.
##### 1. Create dataframe with existing entities and indents
##### 2. Create dataframe with new entities and  indents

In [21]:
class AnalysisQustion(object):
    
    def __init__(self):
        self.df_entity_indent = pd.DataFrame()
        self.model = gensim.models.KeyedVectors.load_word2vec_format('D:/Altimetrik files/CLARA/Entity Extraction/GoogleNews-vectors-negative300.bin',
                                                                     binary=True)        
        self.df_new_entity_indent = pd.DataFrame()
     
    
    # ------------------ Create DataFrame with existing Indents and Entities ---------------------
    def create_df(self, text):
        response = nlu.analyze(text=text.lower(),
                               features=[features.Entities(), features.Keywords()])
        row = -1
        
        for entities in response["entities"]:
            row = row + 1
            self.df_entity_indent.loc[row, "mode"] = "entities" 
            self.df_entity_indent.loc[row, "relevance"] = entities["relevance"] 
            self.df_entity_indent.loc[row, "text"] = entities["text"].strip()
            self.df_entity_indent.loc[row, "type"] = entities["type"].strip()
            self.df_entity_indent.loc[row, "count"] = entities["count"]

        for keywords in response["keywords"]:
            row = row + 1
            self.df_entity_indent.loc[row, "mode"] = "indents"
            self.df_entity_indent.loc[row, "relevance"] = keywords["relevance"] 
            self.df_entity_indent.loc[row, "text"] = keywords["text"].strip()
            self.df_entity_indent.loc[row, "type"] = ''
            self.df_entity_indent.loc[row, "count"] = 0
            
        # ---------- Sort dataframe by Type (Entity type provided by IBM Watson) ----------------
        self.df_entity_indent.sort_values(['type'], ascending=[True])
        
        # ---------- Drop duplicates by Text ----------------------------------------------------
        self.df_entity_indent.drop_duplicates(subset=['text'], keep='first')
        
        
    # ----------------------- Search value in dictionary ----------------------------------------
    def search(self,values, searchFor):
        for k in values:
            for v in values[k]:
                if searchFor in v:
                    return k
        return None
    
    
    # ----------------------- Create DataFrame with new Indents and Entities ---------------------
    def Create_New_Indents_Entities(self, dict_indent):
        df = self.df_entity_indent
        row = 0
        df_row = 0
        
        for row in xrange(len(df)):
            if df.loc[row,'mode'] == 'indents' :
                indent = df.loc[row,'text']
                index = self.search(dict_indent, indent)

                if index == None :
                    similar_word = self.model.most_similar(indent, topn=2)
                    similar = zip(*similar_word)

                    i = 0
                    flag = 0

                    for i in range(len(similar[0])):
                        index = self.search(dict_indent, similar[0][i])
                        if index != None :
                            self.df_new_entity_indent.loc[df_row, "mode"] = "indents"
                            self.df_new_entity_indent.loc[df_row, "text"] = indent 
                            self.df_new_entity_indent.loc[df_row, "Similar to"] = str(dict_indent[index]) 
                            df_row = df_row + 1
                            flag = 1
                        i = i + 1

                    if flag == 0:
                        self.df_new_entity_indent.loc[df_row, "mode"] = "indents"
                        self.df_new_entity_indent.loc[df_row, "text"] = indent 
                        self.df_new_entity_indent.loc[df_row, "Similar to"] = "New Indent"
                        df_row = df_row + 1

            elif df.loc[row,'mode'] == "entities" :
                entity = df.loc[row,'text']
                index = self.search(dict_indent, entity)

                if index == None :
                    similar_word = self.model.most_similar(entity, topn=2)
                    similar = zip(*similar_word)
                    i = 0
                    flag = 0


                    for i in range(len(similar[0])):
                        index = self.search(dict_indent, similar[0][i])
                        if index != None :
                            self.df_new_entity_indent.loc[df_row, "mode"] = "entities"
                            self.df_new_entity_indent.loc[df_row, "text"] = entity 
                            self.df_new_entity_indent.loc[df_row, "Similar to"] = str(dict_indent[index]) 
                            df_row = df_row + 1
                            flag = 1
                        i = i + 1

                    if flag == 0:
                        self.df_new_entity_indent.loc[df_row, "mode"] = "entities"
                        self.df_new_entity_indent.loc[df_row, "text"] = entity 
                        self.df_new_entity_indent.loc[df_row, "Similar to"] = "New Indent"
                        df_row = df_row + 1
            row = row + 1

#### Analyze Questions through Watson 

In [14]:
AnalysisQustion = AnalysisQustion()
AnalysisQustion.create_df('Clara, Which car should i buy?')
AnalysisQustion.df_entity_indent

,mode,relevance,text,type,count
0,entities,0.330000,clara,Person,1.0
1,indents,0.995738,clara,,0.0
2,indents,0.639074,car,,0.0


In [15]:
AnalysisQustion.Create_New_Indents_Entities(dictionary)
AnalysisQustion.df_new_entity_indent

,mode,text,Similar to
0,entities,clara,New Indent
1,indents,clara,New Indent
2,indents,car,New Indent
